<a href="https://colab.research.google.com/github/Sanazzzmi/hybrid-GWO_SVR/blob/main/hybrid_GWO_SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Article idea](https://link.springer.com/content/pdf/10.1007/s11356-020-08792-3.pdf)

[Data](https://www.longpaddock.qld.gov.au/silo/point-data/)

### GWO_SVR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# python implementation of Grey wolf optimization (GWO)
# minimizing rastrigin and sphere function
 
import random
import math    # cos() for Rastrigin
import copy    # array-copying convenience
import sys     # max float
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVR
import sklearn.model_selection
import numpy.random as rd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import numpy.random as rd


file_name='/content/drive/MyDrive/file_name2.csv'
df=pd.read_csv(file_name)
df = df.sort_index()
y=df['et_short_crop']
dff = df.drop('et_short_crop', 1)
X=dff
X_train,X_test,y_train,y_test= train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
#-------fitness functions---------
def fitness_function(position):
    file_name='/content/drive/MyDrive/file_name2.csv'
    df=pd.read_csv(file_name)
    df = df.sort_index()
    y=df['et_short_crop']
    dff = df.drop('et_short_crop', 1)
    X=dff
    X_train,X_test,y_train,y_test= train_test_split(X, y, test_size = 0.2, random_state = 0)

    p0 = position[0]
    p1 = position[1]
    p2 = position[2]
    
    rbf_regressor = svm.SVR(kernel = 'rbf', C = p0 ,epsilon =p1, gamma = p2 ).fit(X_train, y_train)  #svm        
    cv_accuracies = cross_val_score(rbf_regressor,X_test,y_test,cv =3,scoring = 'neg_mean_squared_error') # Taking negated value of MSE
    #To minimize the error rate
    accuracies = cv_accuracies.mean()            
    fitness_value = (1 - accuracies)*100

    return fitness_value

In [ ]:
# wolf class
class wolf:
    def __init__(self, fitness, dim, minx, maxx, seed):
        self.rnd = random.Random(seed)
        self.position = [0.0 for i in range(dim)]

        for i in range(dim):

            self.position[i] = ((maxx - minx) * self.rnd.random() + minx)

        self.fitness = fitness(self.position) # curr fitness
    


In [ ]:
# grey wolf optimization (GWO)
def gwo(fitness, max_iter, n, dim, minx, maxx):
    rnd = random.Random(0)
 
    # create n random wolves
    population = [ wolf(fitness, dim, minx, maxx, i) for i in range(n)]
 
    # On the basis of fitness values of wolves
    # sort the population in asc order
    population = sorted(population, key = lambda temp: temp.fitness)
 
    # best 3 solutions will be called as
    # alpha, beta and gaama
    alpha_wolf, beta_wolf, gamma_wolf = copy.copy(population[: 3])
 
 
    # main loop of gwo
    Iter = 0
    while Iter < max_iter:
 
        # after every 10 iterations
        # print iteration number and best fitness value so far
        if Iter % 10 == 0 and Iter > 1:
            print("Iter = " + str(Iter) + " best fitness = %.3f" % alpha_wolf.fitness)
 
        # linearly decreased from 2 to 0
        a = 2*(1 - Iter/max_iter)
 


        
        # updating each population member with the help of best three members
        for i in range(n):
            A1, A2, A3 = a * (2 * rnd.random() - 1), a * (
              2 * rnd.random() - 1), a * (2 * rnd.random() - 1)
            C1, C2, C3 = 2 * rnd.random(), 2*rnd.random(), 2*rnd.random()
 
            X1 = [0.0 for i in range(dim)]
            X2 = [0.0 for i in range(dim)]
            X3 = [0.0 for i in range(dim)]
            Xnew = [0.0 for i in range(dim)]
            for j in range(dim):
                X1[j] = alpha_wolf.position[j] - A1 * abs(
                  C1 - alpha_wolf.position[j] - population[i].position[j])
                X2[j] = beta_wolf.position[j] - A2 * abs(
                  C2 -  beta_wolf.position[j] - population[i].position[j])
                X3[j] = gamma_wolf.position[j] - A3 * abs(
                  C3 - gamma_wolf.position[j] - population[i].position[j])
                Xnew[j]+= X1[j] + X2[j] + X3[j]
             
            for j in range(dim):
                Xnew[j]/=3.0
            
            
            Xnew = list(map(lambda x: x if x > 0  else 0.01, Xnew))

            # fitness calculation of new solution
            fnew = fitness(Xnew)
 
            # greedy selection
            if fnew < population[i].fitness:
                population[i].position = Xnew
                population[i].fitness = fnew
                 
        # On the basis of fitness values of wolves
        # sort the population in asc order
        population = sorted(population, key = lambda temp: temp.fitness)
 
        # best 3 solutions will be called as
        # alpha, beta and gaama
        alpha_wolf, beta_wolf, gamma_wolf = copy.copy(population[: 3])
         
        Iter+= 1
    # end-while
        
    # returning the best solution.
    
    
    return alpha_wolf.position
          

In [ ]:
# Driver code for rastrigin function

print("\nBegin grey wolf optimization on rastrigin function\n")
dim = 3
fitness = fitness_function
 
 
print("Goal is to minimize Rastrigin's function in " + str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
  print("0, ", end="")
print("0)")
 
num_particles = 100
max_iter =50
 
print("Setting num_particles = " + str(num_particles))
print("Setting max_iter    = " + str(max_iter))
print("\nStarting GWO algorithm\n")
 
 

best_position = gwo(fitness, max_iter, num_particles, dim, 0, 0.1)
 
print("\nGWO completed\n")
print("\nBest solution found:")
print(["%.6f"%best_position[k] for k in range(dim)])
err = fitness(best_position)
print("fitness of best solution = %.6f" % err)
 
print("\nEnd GWO for rastrigin\n")



Begin grey wolf optimization on rastrigin function

Goal is to minimize Rastrigin's function in 3 variables
Function has known min = 0.0 at (0, 0, 0)
Setting num_particles = 100
Setting max_iter    = 50

Starting GWO algorithm

Iter = 10 best fitness = 128.248
Iter = 20 best fitness = 123.411
Iter = 30 best fitness = 121.376
Iter = 40 best fitness = 120.754

GWO completed


Best solution found:
['9.480048', '0.483955', '0.003429']
fitness of best solution = 120.164064

End GWO for rastrigin



In [ ]:
#Apply Optimal Parameters to SVR

svr_regressor= SVR(kernel='rbf', C = best_position[0],epsilon = best_position[1],gamma=best_position[2] )
svr_regressor.fit(X_train,y_train)

y_pred = svr_regressor.predict(X_test)

import numpy as np

accuracies = cross_val_score(svr_regressor, X = X_train, y = y_train, cv = 10)
accuracy_mean= accuracies.mean()
accuracies.std()*100
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE =", rmse)

RMSE = 0.40500065995802903
